In [1]:
from datasets import load_dataset
dataset = load_dataset("json",data_files="/kaggle/input/my-log-summarization-data/my_log_summarization_data2.json")
dataset = dataset['train'].train_test_split(test_size = 0.1)

Generating train split: 0 examples [00:00, ? examples/s]

In [2]:
from transformers import BartTokenizer,BartForConditionalGeneration,Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq

model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

2025-07-02 17:08:09.648105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751476089.842266      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751476089.893351      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [3]:
def preprocess(example):
    inputs = tokenizer(example['text'], truncation=True, padding="max_length", max_length=1024)
    labels = tokenizer(example['summary'],truncation = True,padding="max_length",max_length=128)
    inputs["labels"] = labels["input_ids"]
    return inputs

In [4]:
tokenized = dataset.map(preprocess,batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [5]:
import transformers 
import torch
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    report_to="none",  # disables wandb, tensorboard, etc.
    logging_dir="./logs",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    predict_with_generate=True
)

In [6]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['test'],
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipykernel_19/3226520919.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [7]:
import wandb
wandb.login(key="a1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p6q7r8s9t0")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [8]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.028265
2,No log,0.034612
3,0.107100,0.019636


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=675, training_loss=0.07980533983972338, metrics={'train_runtime': 855.3488, 'train_samples_per_second': 3.157, 'train_steps_per_second': 0.789, 'total_flos': 5851182425702400.0, 'train_loss': 0.07980533983972338, 'epoch': 3.0})

In [9]:
trainer.save_model("custom-bart-summary-overlap")

In [10]:
import re
def duplicate_removal(summary_text):
    sentences = re.split(r'(?<=[.!?])\s+',summary_text.strip())

    seen = set()
    unique_sentences = []

    for sentence in sentences:
        cleaned = sentence.strip()
        if cleaned and cleaned.lower() not in seen:
            seen.add(cleaned.lower())
            unique_sentences.append(cleaned)

    return ' '.join(unique_sentences)

In [11]:
from transformers import pipeline
import re

# Load the models
summary_pipeline = pipeline("summarization", model="custom-bart-summary-overlap")
corrector_pipeline = pipeline("text2text-generation", model="vennify/t5-base-grammar-correction")

def summarize_file(text):
    overlap = 200
    max_chunk = 1000
    #overlapping chunks
    text_chunks = [text[i:i+max_chunk] for i in range(0, len(text), max_chunk - overlap)]

    # Generate raw summary
    raw_summary = ""
    for chunk in text_chunks:
        summary_part = summary_pipeline(chunk, max_length=150, min_length=50, do_sample=False)
        raw_summary += summary_part[0]['summary_text'] + " "
    
    raw_summary = raw_summary.strip()

    # Split summary into sentences using regex
    sentences = re.split(r'(?<=[.!?])\s+', raw_summary)

    # Apply grammar correction to each sentence
    corrected_sentences = []
    for sentence in sentences:
        if sentence.strip():
            corrected = corrector_pipeline(sentence)[0]['generated_text']
            corrected_sentences.append(corrected)

    # Recombine and return the corrected summary
    summary = ' '.join(corrected_sentences)
    dup_removed = duplicate_removal(summary)
    return dup_removed

/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
Device set to use cuda:0


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [12]:
text = '''[2025-06-17 10:00:00] INFO: Unauthorized access attempt to '/opt/data' by user 'admin'
[2025-06-17 10:05:00] INFO: Multiple failed login attempts for user 'alice' detected
[2025-06-17 10:06:00] INFO: Unauthorized access attempt to '/var/log/syslog' by user 'root'
[2025-06-17 10:10:00] ERROR: Cron job 'rotate_logs.sh' executed successfully
[2025-06-17 10:11:00] INFO: Failed login attempt from IP 192.168.1.10 for user 'jdoe'
[2025-06-17 10:14:00] WARNING: Unauthorized access attempt to '/home/user/docs' by user 'jdoe'
[2025-06-17 10:16:00] INFO: Service 'postgresql' restarted by user 'admin'
[2025-06-17 10:18:00] ERROR: Multiple failed login attempts for user 'root' detected
[2025-06-17 10:20:00] WARNING: Unauthorized access attempt to '/opt/data' by user 'admin'
[2025-06-17 10:24:00] DEBUG: Disk space running low on server1
[2025-06-17 10:28:00] WARNING: Disk space running low on server2
[2025-06-17 10:29:00] DEBUG: User 'jdoe' successfully uploaded file 'backup.zip'
[2025-06-17 10:32:00] DEBUG: Disk space running low on /dev/sdb2
[2025-06-17 10:33:00] ERROR: New SSH key added for user 'bob'
[2025-06-17 10:34:00] ERROR: User 'root' successfully uploaded file 'config.yaml'
[2025-06-17 10:35:00] INFO: Disk space running low on server2
[2025-06-17 10:39:00] ERROR: Connection timeout to database server at 198.51.100.2
[2025-06-17 10:40:00] DEBUG: Disk space running low on /dev/sdb2
[2025-06-17 10:45:00] INFO: Connection timeout to database server at 192.168.1.10
[2025-06-17 10:47:00] WARNING: Failed login attempt from IP 10.0.0.15 for user 'system'
[2025-06-17 10:50:00] WARNING: Disk space running low on /dev/sdb2
[2025-06-17 10:53:00] DEBUG: Backup completed for /etc/passwd
[2025-06-17 10:57:00] DEBUG: Service 'postgresql' restarted by user 'jdoe'
[2025-06-17 11:01:00] ERROR: Connection timeout to database server at 10.0.0.15
[2025-06-17 11:02:00] DEBUG: System update completed successfully
[2025-06-17 11:05:00] INFO: System update completed successfully
[2025-06-17 11:09:00] ERROR: New SSH key added for user 'system'
[2025-06-17 11:11:00] DEBUG: Backup completed for /opt/data
[2025-06-17 11:16:00] ERROR: Backup completed for /etc/passwd
[2025-06-17 11:21:00] WARNING: Multiple failed login attempts for user 'bob' detected
[2025-06-17 11:23:00] WARNING: Multiple failed login attempts for user 'guest' detected
[2025-06-17 11:26:00] WARNING: System update completed successfully
[2025-06-17 11:31:00] WARNING: Service 'postgresql' restarted by user 'root'
[2025-06-17 11:32:00] WARNING: Backup completed for /opt/data
[2025-06-17 11:36:00] ERROR: Memory usage at 70% on server '/dev/sda1'
[2025-06-17 11:41:00] WARNING: Service 'postgresql' restarted by user 'guest'
[2025-06-17 11:45:00] ERROR: Service 'nginx' restarted by user 'alice'
[2025-06-17 11:47:00] WARNING: Service 'nginx' restarted by user 'guest'
[2025-06-17 11:48:00] WARNING: Cron job 'cleanup_temp.sh' executed successfully
[2025-06-17 11:49:00] ERROR: Failed login attempt from IP 203.0.113.45 for user 'bob'
[2025-06-17 11:52:00] INFO: Cron job 'backup_daily.sh' executed successfully
[2025-06-17 11:53:00] INFO: Backup completed for /opt/data
[2025-06-17 11:56:00] WARNING: Connection timeout to database server at 203.0.113.45
[2025-06-17 11:57:00] INFO: Cron job 'rotate_logs.sh' executed successfully
[2025-06-17 11:58:00] WARNING: Cron job 'cleanup_temp.sh' executed successfully
[2025-06-17 12:00:00] WARNING: Multiple failed login attempts for user 'jdoe' detected
[2025-06-17 12:02:00] ERROR: New SSH key added for user 'bob'
[2025-06-17 12:07:00] DEBUG: Failed login attempt from IP 192.168.1.10 for user 'system'
[2025-06-17 12:09:00] DEBUG: Multiple failed login attempts for user 'bob' detected
[2025-06-17 12:13:00] DEBUG: Disk space running low on /dev/sda1
[2025-06-17 12:14:00] DEBUG: Connection timeout to database server at 10.0.0.15
[2025-06-17 12:16:00] ERROR: Memory usage at 87% on server '/dev/sda1'
[2025-06-17 12:19:00] INFO: Unauthorized access attempt to '/etc/passwd' by user 'system'
[2025-06-17 12:21:00] INFO: Service 'redis' restarted by user 'alice'
[2025-06-17 12:26:00] WARNING: Memory usage at 79% on server '/dev/sdb2'
[2025-06-17 12:30:00] WARNING: Unauthorized access attempt to '/var/log/syslog' by user 'guest'
[2025-06-17 12:34:00] INFO: Failed login attempt from IP 203.0.113.45 for user 'root'
[2025-06-17 12:36:00] INFO: Service 'postgresql' restarted by user 'system'
[2025-06-17 12:40:00] INFO: Memory usage at 80% on server 'server1'
[2025-06-17 12:44:00] INFO: Connection timeout to database server at 203.0.113.45
[2025-06-17 12:47:00] ERROR: Unauthorized access attempt to '/etc/passwd' by user 'root'
[2025-06-17 12:48:00] WARNING: Failed login attempt from IP 203.0.113.45 for user 'jdoe'
[2025-06-17 12:51:00] ERROR: Service 'nginx' restarted by user 'root'
[2025-06-17 12:52:00] ERROR: Backup completed for /opt/data
[2025-06-17 12:56:00] INFO: New SSH key added for user 'bob'
[2025-06-17 12:58:00] WARNING: Failed login attempt from IP 203.0.113.45 for user 'system'
[2025-06-17 13:01:00] DEBUG: Failed login attempt from IP 198.51.100.2 for user 'guest'
[2025-06-17 13:06:00] ERROR: Unauthorized access attempt to '/etc/passwd' by user 'root'
[2025-06-17 13:08:00] ERROR: Memory usage at 71% on server 'server1'
[2025-06-17 13:12:00] WARNING: Multiple failed login attempts for user 'system' detected
[2025-06-17 13:16:00] INFO: Multiple failed login attempts for user 'bob' detected
[2025-06-17 13:21:00] INFO: Backup completed for /var/log/syslog
[2025-06-17 13:26:00] DEBUG: Connection timeout to database server at 203.0.113.45
[2025-06-17 13:28:00] ERROR: User 'root' successfully uploaded file 'backup.zip'
[2025-06-17 13:29:00] DEBUG: Memory usage at 96% on server '/dev/sdb2'
[2025-06-17 13:31:00] INFO: Failed login attempt from IP 198.51.100.2 for user 'system'
[2025-06-17 13:32:00] INFO: Connection timeout to database server at 198.51.100.2
[2025-06-17 13:35:00] INFO: Disk space running low on server1
[2025-06-17 13:38:00] INFO: Cron job 'backup_daily.sh' executed successfully
[2025-06-17 13:39:00] DEBUG: Cron job 'rotate_logs.sh' executed successfully
[2025-06-17 13:42:00] INFO: Multiple failed login attempts for user 'system' detected
[2025-06-17 13:47:00] DEBUG: Failed login attempt from IP 10.0.0.15 for user 'alice'
[2025-06-17 13:50:00] DEBUG: Unauthorized access attempt to '/opt/data' by user 'jdoe'
[2025-06-17 13:54:00] DEBUG: User 'alice' successfully uploaded file 'error.log'
[2025-06-17 13:56:00] INFO: Service 'redis' restarted by user 'alice'
[2025-06-17 14:00:00] ERROR: Failed login attempt from IP 198.51.100.2 for user 'root'
[2025-06-17 14:05:00] INFO: System update completed successfully
[2025-06-17 14:10:00] DEBUG: Memory usage at 74% on server 'server1'
[2025-06-17 14:13:00] WARNING: Disk space running low on /dev/sdb2
[2025-06-17 14:18:00] WARNING: Cron job 'rotate_logs.sh' executed successfully
[2025-06-17 14:22:00] INFO: Service 'mysql' restarted by user 'bob'
[2025-06-17 14:25:00] DEBUG: Unauthorized access attempt to '/etc/passwd' by user 'system'
[2025-06-17 14:30:00] ERROR: Memory usage at 90% on server '/dev/sdb2'
[2025-06-17 14:33:00] INFO: Backup completed for /etc/passwd
[2025-06-17 14:35:00] DEBUG: Connection timeout to database server at 198.51.100.2
[2025-06-17 14:36:00] ERROR: Multiple failed login attempts for user 'guest' detected
[2025-06-17 14:41:00] DEBUG: Multiple failed login attempts for user 'system' detected
[2025-06-17 14:42:00] WARNING: User 'bob' successfully uploaded file 'report.log'
[2025-06-17 14:45:00] INFO: System update completed successfully
[2025-06-17 14:50:00] INFO: Connection timeout to database server at 192.168.1.10'''

summary = summarize_file(text)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [13]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

In [14]:
from transformers import pipeline
from evaluate import load

In [15]:
pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e410087b121609e04daa5ab68ff1da59ae11a7e632453ef592d0db2c5b0ddf49
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [16]:
reference_summary = "The system logs indicate a series of significant security and operational events. Numerous unauthorized access attempts were made to critical files and directories such as `/opt/data`, `/etc/passwd`, and `/var/log/syslog`, primarily by users including 'admin', 'root', 'jdoe', 'system', and 'guest'. There was a high volume of failed login attempts, with multiple repeated failures detected for users like 'alice', 'bob', 'jdoe', 'guest', and 'system'. Several successful and potentially suspicious file uploads were carried out by users such as 'root', 'alice', and 'bob', involving sensitive files like `backup.zip`, `config.yaml`, and `error.log`. Additionally, new SSH keys were added for users 'bob' and 'system', which may indicate unauthorized access or configuration changes. System services including PostgreSQL, Nginx, Redis, and MySQL were restarted multiple times by different users, which could reflect troubleshooting efforts or malicious activities. A number of cron jobs (`rotate_logs.sh`, `cleanup_temp.sh`, and `backup_daily.sh`) were executed successfully, and several backup operations were completed for system-critical files. However, the system showed signs of performance degradation, with repeated warnings about low disk space on `/dev/sdb2`, `/dev/sda1`, and various servers, as well as high memory usage reaching up to 96%. Connection timeouts were frequently reported when attempting to access database servers at several IP addresses, suggesting potential network issues or overloaded systems. While system updates were completed successfully on multiple occasions, the combination of security alerts, service restarts, resource exhaustion, and connectivity failures suggests the environment may be under stress or potentially compromised."
rouge = load("rouge")
results = rouge.compute(predictions=[summary],references=[reference_summary])

print("\nGenerated Summary:")
print(summary)
print("\n Rouge Scores:")
for metric, score in results.items():
    print(f"{metric}:{score:.4f}")


Generated Summary:
Unauthorized access attempt by 'admin' policymakers. Failed login attempts by 'alice' detected. Unauthorized access attempt to 'var/log/syslog' by ' Cron job 'rotate_logs. Executed. Failed Login attempt by jdoe' created. Unauthorized access by 'jdoe-policy. Low disk space warning. Service 'postgresql' restarted. Multiple failed login attempts for 'root's detected. Database connection by ' admin' Petertodd. Low disk space warnings. File upload by 'jdoe' successfully uploaded. Database connection timeout. File deletion by 'root' Low diskspace warning. Failed login attempt by 'system' policymakers. Backup completed. SSH key added for 'bob'utable. System update completed. SSH key added for 'system' coalesce. Backup completion. Failed login attempt by 'bob' detected. Failed Login attempts by 'guest' detected by 'jdoe' Backup completed. Memory usage at 70% on server '/dev/sda1'utable. Service 'postgradesql' completed. Service service 'nginx' restarted. Cron job 'rotate_lo